In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import timedelta, datetime, date


fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)

# Parameters
NUM_CUSTOMERS = 100
NUM_PRODUCTS = 50
NUM_ORDERS = 1000
INCLUDE_NOISE = True  # set False for squeaky-clean data


In [ ]:
#customer table

customer = pd.DataFrame({
        "customer_id" : range(1, NUM_CUSTOMERS + 1),
        "first_name": [fake.first_name() for _ in range(NUM_CUSTOMERS)],
        "last_name": [fake.last_name() for _ in range(NUM_CUSTOMERS)],
        "email": [fake.email() for _ in range(NUM_CUSTOMERS)],
        "join_date": [fake.date_between(start_date="-1y", end_date="today") for _ in range(NUM_CUSTOMERS)]

})

In [ ]:
product_categories = ['Electronics', 'Clothing', 'Home Appliances', 'Books', 'Sports Equipment']
products = pd.DataFrame({
   'product_id': range(1, NUM_PRODUCTS + 1),
   'product_name': [fake.word().capitalize() + ' ' + random.choice(['Pro', 'Plus', 'Lite', 'Max']) for _ in range(NUM_PRODUCTS)],
   'category': [random.choice(product_categories) for _ in range(NUM_PRODUCTS)],
   'price': [round(random.uniform(10.0, 500.0), 2) for _ in range(NUM_PRODUCTS)]
})

In [ ]:
orders = []
for i in range(1, NUM_ORDERS + 1):
    order_date = fake.date_between(start_date='-1y', end_date='today')
    # ship 0–7 days after order
    ship_lag = np.random.choice(range(0, 8), p=[0.10, 0.20, 0.20, 0.18, 0.12, 0.10, 0.06, 0.04])
    ship_date = order_date + timedelta(days=int(ship_lag))

    # deliver 1–10 days after ship, with small chance still in transit (NULL)
    if np.random.rand() < 0.93:
        delivery_lag = np.random.randint(1, 11)
        delivery_date = ship_date + timedelta(days=int(delivery_lag))
    else:
        delivery_date = None  # still in transit / pending

    order = {
        'order_id': i,
        'customer_id': random.randint(1, NUM_CUSTOMERS),
        'product_id': random.randint(1, NUM_PRODUCTS),
        'quantity': random.randint(1, 5),
        'order_date': order_date,
        'ship_date': ship_date,
        'delivery_date': delivery_date
    }
    orders.append(order)

orders_df = pd.DataFrame(orders)

# Add total_price = quantity * price (join with products)
orders_df = orders_df.merge(products[['product_id', 'price']], on='product_id', how='left')
orders_df['total_price'] = (orders_df['quantity'] * orders_df['price']).round(2)


In [ ]:
if INCLUDE_NOISE:
    # 1) Duplicate a small slice of orders (about 1%)
    dupes = orders_df.sample(max(5, int(len(orders_df) * 0.01)), random_state=42)
    orders_df = pd.concat([orders_df, dupes], ignore_index=True)

    # 2) Inject some NULLs
    orders_df.loc[orders_df.sample(8, random_state=42).index, 'customer_id'] = None   # orphan orders
    customer.loc[customer.sample(5, random_state=42).index, 'email'] = None         # missing emails

    # 3) Inconsistent category casing (lowercase some)
    idx = products.sample(max(5, int(len(products)*0.10)), random_state=42).index
    products.loc[idx, 'category'] = products.loc[idx, 'category'].str.lower()

    # 4) A few outliers: zero/negative quantities & prices
    q_idx = orders_df.sample(2, random_state=42).index
    orders_df.loc[q_idx, 'quantity'] = [0, -2]  # bad quantities
    p_idx = products.sample(2, random_state=42).index
    products.loc[p_idx, 'price'] = [0.0, -7.99]  # bad prices

    # Re-merge price after product price noise and recompute total
    orders_df = orders_df.drop(columns=['price'])
    orders_df = orders_df.merge(products[['product_id', 'price']], on='product_id', how='left')
    orders_df['total_price'] = (orders_df['quantity'] * orders_df['price']).round(2)

In [ ]:
# -------------------------------------------
# Save as CSVs (for SQL)
# -------------------------------------------
customer.to_csv('customers.csv', index=False)
products.to_csv('products.csv', index=False)
orders_df.to_csv('orders.csv', index=False)